# Random data - synthetic demo

This is a very simple demonstration of how to use `SimilaritySearch.jl`. The API correspond to version `0.8`

In [1]:
using Pkg
Pkg.activate(".")
Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.8.10")
])

using SimilaritySearch

  Activating project at `~/Research/SimilaritySearchDemos/synthetic`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Research/SimilaritySearchDemos/synthetic/Project.toml`
  No Changes to `~/Research/SimilaritySearchDemos/synthetic/Manifest.toml`


# A random dataset
Let us define a dataset of 8-dimensions and $10^5$ elements. Each object is a column. The matrix needs to be wrapped as a database since `SimilaritySearch` is distance agnostic and objects can be any representation. The matrix is not copied.


In [2]:
n = 100_000
M = randn(Float32, 8, n)
db = MatrixDatabase(M)

MatrixDatabase{Matrix{Float32}}(Float32[-0.12045016 -0.53475857 … -0.4273328 1.1824491; 1.1121974 0.32277134 … 0.45995116 2.13024; … ; -0.57610315 0.8922343 … -0.39629176 -0.06793562; -0.88367796 -0.6474708 … 0.6428879 -0.087060295])

The database object mimics a vector of elements

In [3]:
length(db), eltype(db), typeof(db[1])

(100000, AbstractVector{Float32}, SubArray{Float32, 1, Matrix{Float32}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true})

The `SubArray` that results of `typeof(db[1])` means that each object is a column's `view`, and therefore there is no extra memory allocations.

# Index construction

An index is defined as follows

In [4]:
dist = SqL2Distance()
G = SearchGraph(; db, dist)

SearchGraph{SqL2Distance, MatrixDatabase{Matrix{Float32}}, BeamSearch}
  dist: SqL2Distance SqL2Distance()
  db: MatrixDatabase{Matrix{Float32}}
  links: Array{Vector{Int32}}((0,))
  locks: Array{Base.Threads.SpinLock}((0,))
  hints: Array{Int32}((0,)) Int32[]
  search_algo: BeamSearch
  neighborhood: SimilaritySearch.Neighborhood
  verbose: Bool true


The `SearchGraph` index has an incremental construction that contains a list of callbacks that are called at exponential steps. By default it uses `OptimizeParameters(kind=ParetoRecall())` such that our index try to optimize jointly search speed and recall. It is also possible to optimize for a minimum recall with `MinRecall(0.9)` for a construction that will try to reach 0.9 of recall (using the same dataset as gold standard).

The index is defined, it needs to be constructed as follows (please note that the construction can output a lot of information):

In [5]:
# index!(G, callbacks=SearchGraphCallbacks(hyperparameters=OptimizeParameters(kind=MinRecall(0.9))))
index!(G)
IJulia.clear_output()

0

# Searching

Searching can be performed with methods `search` and `searchbatch`. Both are pretty similar, the first one solves a single query and the second method solves a batch of queries. 

In [6]:
I, D = searchbatch(G, MatrixDatabase(rand(8, 3)), 10)
size(I), size(D)

((10, 3), (10, 3))

It returns two matrices of size $10 \times 3$ (10nn of the three given queries). Please note that our dataset is composed of Vector of Float32 elements and we are asking for Float64 vector queries. This is allowed due to the automatic specialization of Julia, but it may impact on the performance (due to SIMD ops.)

A similar way to search is using an array of queries

In [7]:
I, D = searchbatch(G, [rand(Float32, 8) for i in 1:3], 10)
size(I), size(D)

((10, 3), (10, 3))

Note: Querying directly for rand(8, 3) will perform unexpected results. Note: the cannonical way to perform queries `searchbatch` is the first one (wrapping the queryset with a MatrixDatabase) and the second form should be used only for fast scripting since it always.

## Single queries
The function `search` solves single queries, specified and stored with a `KnnResult` struct.

In [8]:
res, cost = search(G, rand(Float32, 8), KnnResult(10))

(res = KnnResult(Int32[34783, 43265, 4903, 68840, 74177, 12783, 38107, 44916, 1709, 42222], Float32[0.20727837, 0.40561062, 0.45936117, 0.4621851, 0.4996728, 0.5269037, 0.6028715, 0.6450891, 0.6595993, 0.6710913], 10), cost = 298)

The function `search` returns the struct passed as argument (`KnnResult(10)`) and the number of distance evaluations performed to solve it.

The `res` object has several related functions, but internally, it contains identifiers and distances. The identifiers are indexes in the database to access the retrieved nearest neighbors; and its respective distances to the query. `KnnResult` objects can be iterated at accessed by position.

In [9]:
display("text/markdown", """

- Nearest neighbor pair: `$(first(res))`
- argmin: $(argmin(res)), minimum: $(minimum(res))
- argmax: $(argmax(res)), maximum: $(maximum(res))
- 1nn: $(getpair(res, 1)), 2nn: $(getpair(res, 1))
- knns: $(res.id)
- dists: $(res.dist)
    

The `KnnResult` is a priority queue that stores at most `k` pairs.
You can modify it using `push!`, `pop!` and `popfirst!`

""")

display((:popfirst! => popfirst!(res), :res => res, length => length(res)))
display((:pop! => pop!(res), :res => res, :length => length(res)))
push!(res, 1, 0.0)
push!(res, 2, 1e6)
display(:after_push! => res)
display("text/markdown", "### You can also iterate the result set and access to the indexed dataset")
for (i, (id, dist)) in enumerate(res)
    println(i => (dist, G[id]))
end





- Nearest neighbor pair: `34783 => 0.20727837f0`
- argmin: 34783, minimum: 0.20727837
- argmax: 42222, maximum: 0.6710913
- 1nn: 34783 => 0.20727837f0, 2nn: 34783 => 0.20727837f0
- knns: Int32[34783, 43265, 4903, 68840, 74177, 12783, 38107, 44916, 1709, 42222]
- dists: Float32[0.20727837, 0.40561062, 0.45936117, 0.4621851, 0.4996728, 0.5269037, 0.6028715, 0.6450891, 0.6595993, 0.6710913]
    

The `KnnResult` is a priority queue that stores at most `k` pairs.
You can modify it using `push!`, `pop!` and `popfirst!`



(:popfirst! => (34783 => 0.20727837f0), :res => KnnResult(Int32[43265, 4903, 68840, 74177, 12783, 38107, 44916, 1709, 42222], Float32[0.40561062, 0.45936117, 0.4621851, 0.4996728, 0.5269037, 0.6028715, 0.6450891, 0.6595993, 0.6710913], 10), length => 9)

(:pop! => (42222 => 0.6710913f0), :res => KnnResult(Int32[43265, 4903, 68840, 74177, 12783, 38107, 44916, 1709], Float32[0.40561062, 0.45936117, 0.4621851, 0.4996728, 0.5269037, 0.6028715, 0.6450891, 0.6595993], 10), :length => 8)

:after_push! => KnnResult(Int32[1, 43265, 4903, 68840, 74177, 12783, 38107, 44916, 1709, 2], Float32[0.0, 0.40561062, 0.45936117, 0.4621851, 0.4996728, 0.5269037, 0.6028715, 0.6450891, 0.6595993, 1.0f6], 10)

### You can also iterate the result set and access to the indexed dataset

1 => (0.0f0, Float32[-0.12045016, 1.1121974, -1.2203869, -0.23189978, 0.3345606, 0.6638271, -0.57610315, -0.88367796])
2 => (0.40561062f0, Float32[-0.075216055, 0.3513729, 0.81024456, 1.0519006, 0.75267243, 0.092860155, 1.0022776, 0.14793374])
3 => (0.45936117f0, Float32[0.48839122, 0.6536945, 0.8076429, 0.35507175, 0.45182833, 0.4276768, 0.55495995, 0.06989036])
4 => (0.4621851f0, Float32[-0.20702612, 0.59180117, 0.7452996, 0.7756401, 0.3880822, 0.31119123, 1.0533082, 0.23561564])
5 => (0.4996728f0, Float32[0.012422479, 0.37038645, 1.0395277, 1.1880285, 0.5033326, 0.27784678, 0.18746497, 0.11635025])
6 => (0.5269037f0, Float32[-0.14843248, 0.63722026, 0.8738692, 0.27568966, 0.6324766, 0.4627273, 0.89337206, 0.08610707])
7 => (0.6028715f0, Float32[0.11728806, 0.54548043, 0.4672482, 0.5921352, 0.90494245, 0.110077135, 0.4441372, 0.48396742])
8 => (0.6450891f0, Float32[0.4371052, 0.5013931, 1.028366, 0.6856857, 0.6675062, 0.58435124, -0.03640411, 0.23755834])
9 => (0.6595993f0, Float32[-